In [1]:
import sys
import json
import cobra
import cplex
import cobrakbase
import escher
from escher import Builder
from optlang.symbolics import Zero, add
#Put the path to ModelSEEDpy on your machine here
sys.path.append("/Users/chenry/code/ModelSEEDpy")
#import modelseedpy.fbapkg
from modelseedpy import MSPackageManager

cobrakbase 0.2.7


In [2]:
file = open('plant_map.json',)
plantmap = file.read()
builder = Builder(map_json=plantmap)
builder.reaction_scale=[
        { "type": 'value', "value": 0, "color": '#dcdcdc', "size": 10 },
        { "type": 'value', "value": 0.000001, "color": '#9696ff', "size": 25 },
        { "type": 'value', "value": 1, "color": '#ff0000', "size": 35 },
        { "type": 'value', "value": 20, "color": '#209123', "size": 50 },
    ]
builder

Builder(reaction_scale=[{'type': 'value', 'value': 0, 'color': '#dcdcdc', 'size': 10}, {'type': 'value', 'valu…

In [3]:
buildertwo = Builder(map_json=plantmap)
buildertwo.reaction_scale=[
        { "type": 'value', "value": 0, "color": '#dcdcdc', "size": 10 },
        { "type": 'value', "value": 0.000001, "color": '#9696ff', "size": 25 },
        { "type": 'value', "value": 1, "color": '#ff0000', "size": 35 },
        { "type": 'value', "value": 20, "color": '#209123', "size": 50 },
    ]
buildertwo

Builder(reaction_scale=[{'type': 'value', 'value': 0, 'color': '#dcdcdc', 'size': 10}, {'type': 'value', 'valu…

In [4]:
#Uncomment the tissue datastructure for the model you want to run
#Single tissue model:
#tissues = ["Mature_leaf"]
#Two tissue model:
tissues = ["Mature_leaf","GermEmbryo_2_DAI"]
#16 tissue model:
#tissues = ["Mature_leaf","Zone_1","Zone_2","Zone_3","GermEmbryo_2_DAI","Pericarp_Aleurone_27_DAP","Endosperm_Crown_27_DAP","Endosperm_12_DAP","Endosperm_10_DAP","Endosperm_8_DAP","Embryo_38_DAP","Embryo_20_DAP","root_Stele","root_Cortex","root_EZ","root_MZ"]
kbase_api = cobrakbase.KBaseAPI()
model = kbase_api.get_from_ws("C4_HS_Zmays_Model",25427)
model.solver = 'optlang-cplex'
#Setting model to complete media
#media = kbase_api.get_from_ws("Carbon-D-Glucose","KBaseMedia")
pkgmgr = MSPackageManager.get_pkg_mgr(model,1)
pkgmgr.getpkg("KBaseMediaPkg",1).build_package(None)

#Reading reaction protein abundances
data = ""
with open('/Users/chenry/code/ModelSEEDpy/examples/ReactionProtein.txt', 'r') as file:
    data = file.read()
lines = data.split("\n")
reaction_proteins = {}
headers = None
for line in lines:
    if headers == None:
        headers = line.split("\t")
    else:
        array = line.split("\t")
        for i in range(2, len(array)):
            if headers[i] not in reaction_proteins:
                reaction_proteins[headers[i]] = dict()
            reaction_proteins[headers[i]][array[0]] = abs(float(array[i]))

#Reading reaction kcat values
data = ""
with open('/Users/chenry/code/ModelSEEDpy/examples/KCats.txt', 'r') as file:
    data = file.read()
lines = data.split("\n")
reaction_kcat = {}
headers = None
for line in lines:
    if headers == None:
        headers = line.split("\t")
    else:
        array = line.split("\t")
        reaction_kcat[array[0]] = abs(float(array[1]))

#Reading measured reaction fluxes
data = ""
with open('/Users/chenry/code/ModelSEEDpy/examples/MeasuredReaction.txt', 'r') as file:
    data = file.read()
lines = data.split("\n")
reaction_measures = {}
headers = None
for line in lines:
    if headers == None:
        headers = line.split("\t")
    else:
        array = line.split("\t")
        if array[1] not in reaction_measures:
                reaction_measures[array[1]] = dict()
        rxnid = array[0]
        reaction_measures[array[1]][rxnid] = abs(float(array[2]))

biomass_fluxes = {
    "root_MZ" : 0.049925,
    "root_EZ" : 0.049925,
    "root_Cortex" : 0.049925,
    "root_Stele" : 0.049925,
    "Embryo_20_DAP" : 0.015591667,
    "Embryo_38_DAP" : 0.015591667,
    "Endosperm_8_DAP" : 0.010208333,
    "Endosperm_10_DAP" : 0.010208333,
    "Endosperm_12_DAP" : 0.010208333,
    "Endosperm_Crown_27_DAP" : 0.010208333,
    "Pericarp_Aleurone_27_DAP" : 0.001529167,
    "GermEmbryo_2_DAI" : 0.015591667,
    "Zone_1" : 0.006395833,
    "Zone_2" : 0.006395833,
    "Zone_3" : 0.006395833,
    "Mature_leaf" : 0.006395833
}

protein_fraction = {
    "root_MZ" : 0.0948,
    "root_EZ" : 0.0948,
    "root_Cortex" : 0.0948,
    "root_Stele" : 0.0948,
    "Embryo_20_DAP" : 0.0948,
    "Embryo_38_DAP" : 0.0948,
    "Endosperm_8_DAP" : 0.0948,
    "Endosperm_10_DAP" : 0.0948,
    "Endosperm_12_DAP" : 0.0948,
    "Endosperm_Crown_27_DAP" : 0.0948,
    "Pericarp_Aleurone_27_DAP" : 0.0948,
    "GermEmbryo_2_DAI" : 0.0948,
    "Zone_1" : 0.0948,
    "Zone_2" : 0.0948,
    "Zone_3" : 0.0948,
    "Mature_leaf" : 0.0948
}

rxn = model.reactions.get_by_id("bio1")
rxn.upper_bound = 0.006395833
rxn.lower_bound = 0.006395833
sol=model.optimize()
builder.model = model
builder.reaction_data = sol.fluxes

modellist = []
count = 0
base_model = None
for tissue in tissues:
    print("Tissue:"+tissue)
    #Safe cloning method
    clone_model = cobra.io.json.from_json(cobra.io.json.to_json(model))
    #Applying flexible biomass constraints
    pkgmgr = MSPackageManager.get_pkg_mgr(clone_model,1)
    protein = protein_fraction[tissue]
    protein_flux = -1*(0.2-protein)
    pkgmgr.getpkg("FlexibleBiomassPkg",1).build_package({"bio_rxn_id":"bio1","use_rna_class":[-0.75,0.75],
        "use_dna_class":[-0.75,0.75],
        "use_protein_class":[protein_flux,protein_flux],
        "use_energy_class":[-0.1,0.1]})
    #Adding tissue specific proteome constraints
    package_params = {
        "rxn_proteome":reaction_proteins[tissue],
        "flux_values":reaction_measures[tissue],
        "prot_coef" : protein_fraction[tissue],#Set to the fraction of the cell that is protein
        "totalflux" : 1,#Set to one if we're fitting flux magnitude rather than actual flux to flux measurements
        "kcat_coef" : 0.02,#kapp = kcat_coef * kcat
        "set_objective":0
    }
    #Only base model fits kcat values
    if count == 0:
        package_params["kcat_values"] = reaction_kcat
    pkgmgr.getpkg("ProteomeFittingPkg",1).build_package(package_params)
    #Constraining biomass flux to reference value
    rxn = clone_model.reactions.get_by_id("bio1")
    rxn.upper_bound = biomass_fluxes[tissue]
    rxn.lower_bound = biomass_fluxes[tissue]
    if count == 0:
        base_model = clone_model
    else:
        modellist.append(clone_model)
    count += 1

#Coefficient for kcat fitting:
kcat_fit = 10
#Coefficient for flux fitting:
flux_fit = 1000
#Coefficient for kinetic fitting:
kinetic_fit = 1
#Pulling based model package manager
pkgmgr = MSPackageManager.get_pkg_mgr(base_model,1)
#Merging all models together
if count > 1:
    pkgmgr.getpkg("ProblemReplicationPkg",1).build_package({"models":modellist,"shared_variable_packages":{pkgmgr.getpkg("ProteomeFittingPkg"):["kapp"]}})
#Creating combined objective function
objvars = []
#Adding base model kfit,kvfit,and vfit to objective
for rxnid in pkgmgr.getpkg("FluxFittingPkg").variables["vfit"]:
    objvars.append(flux_fit * pkgmgr.getpkg("FluxFittingPkg").variables["vfit"][rxnid] ** 2)
for rxnid in pkgmgr.getpkg("ProteomeFittingPkg").variables["kvfit"]: 
    objvars.append(kinetic_fit * pkgmgr.getpkg("ProteomeFittingPkg").variables["kvfit"][rxnid] ** 2)
for rxnid in pkgmgr.getpkg("ProteomeFittingPkg").variables["kfit"]: 
    objvars.append(kcat_fit * pkgmgr.getpkg("ProteomeFittingPkg").variables["kfit"][rxnid] ** 2)
#Adding sub model kvfit,and vfit to objective
count = 0
for submodel in modellist:
    submgr = MSPackageManager.get_pkg_mgr(submodel,1)
    for rxnid in submgr.getpkg("ProteomeFittingPkg").variables["kvfit"]:
        varname = submgr.getpkg("ProteomeFittingPkg").variables["kvfit"][rxnid].name
        objvars.append(kinetic_fit * pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count)][varname] ** 2)
    for rxnid in submgr.getpkg("FluxFittingPkg").variables["vfit"]:
        varname = submgr.getpkg("FluxFittingPkg").variables["vfit"][rxnid].name
        objvars.append(flux_fit * pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count)][varname] ** 2)
    count += 1
#Setting the objective
base_model.objective = base_model.problem.Objective(add(objvars), direction="min", sloppy=True)

#Printing LP file
with open('ProteomeDrivenModeling.lp', 'w') as out:
    out.write(str(base_model.solver))

sol=base_model.optimize()
buildertwo.model = base_model
buildertwo.reaction_data = sol.fluxes

Tissue:Mature_leaf
Tissue:GermEmbryo_2_DAI


In [5]:
builderthree = Builder(map_json=plantmap)
builderthree.reaction_scale=[
        { "type": 'value', "value": 0, "color": '#dcdcdc', "size": 10 },
        { "type": 'value', "value": 0.000001, "color": '#9696ff', "size": 25 },
        { "type": 'value', "value": 1, "color": '#ff0000', "size": 35 },
        { "type": 'value', "value": 20, "color": '#209123', "size": 50 },
    ]
#Visualizing the fit
fit = {}
for rxnid in pkgmgr.getpkg("FluxFittingPkg").variables["vfit"]:
    fit[rxnid] = pkgmgr.getpkg("FluxFittingPkg").variables["vfit"][rxnid].primal
builderthree.reaction_data = fit
builderthree

Builder(reaction_data={'rxn00102_d1': 3.188478221778222e-05, 'rxn20508_d1': 3.188478221778222e-05, 'rxn00249_d…